Beta-geometric/Negative Binomial Distribution Model (BG/NBD) - Forecasting Individual-Level Repeat-Buying

Note to user interested in using this model:
1) The data being modeled are counts of relatively homogeneous units (e.g., CDs). This model must not be used to model dollar sales or counts of products that are not very similar (e.g., the number of products purchased at Amazon.com where the units include such disparate items as books, electronic equipment, lawn furniture, and so on).
2) Use of the shifted-geometric/geometric distributions implies that the *modal trial quantity* is 1 unit and that the *modal* number of units purchased in subsequent weeks, conditional on being a “possible repeat buyer”, is 0. If this is not that case, it will be necessary to change the underlying model structure. For example, the shifted beta-geometric model of trial counts could be replaced by the truncated or shifted NBD (which can have a mode away from 1). Similarly, the beta-geometric repeat purchasing distribution could be replaced by the NBD (which can have a non-zero mode).
3) A number of other assumptions made, implicitly or explicitly, in the paper should also be acknowledged and taken into account (e.g., the independence of quantity decisions across transactions).

In [1]:
import polars as pl
import numpy as np
import altair as alt
from utils import ChartTemp, layered_line_prop

from scipy.optimize import minimize
from scipy.stats import chi2

from great_tables import GT #, style, loc, md
from IPython.display import display_markdown

In [2]:
CDNOW_master = (
    pl.scan_csv(source = 'data/CDNOW/CDNOW_master.csv', 
                has_header=False, 
                separator=',', 
                schema={'CustID': pl.Int32,     # customer id
                        'Date': pl.String,      # transaction date
                        'Quant': pl.Int16,      # number of CDs purchased
                        'Spend': pl.Float64})   # dollar value (excl. S&H)
    .with_columns(pl.col('Date').str.to_date("%Y%m%d"))
    .with_columns((pl.col('Date') - pl.date(1996,12,31)).dt.total_days().cast(pl.UInt16).alias('PurchDay'))
    .with_columns((pl.col('Spend')*100).round(0).cast(pl.Int64).alias('Spend Scaled'))
    .group_by('CustID', 'Date')
    .agg(pl.col('*').exclude('PurchDay').sum(), pl.col('PurchDay').max()) # Multiple transactions by a customer on a single day are aggregated into one
    .sort('CustID', 'Date')
    .with_columns((pl.col("CustID").cum_count().over("CustID") - 1).cast(pl.UInt16).alias("DoR"))  # DoR = Depth of Repeat ('Transaction' time: starts with 0 as trial, 1 as 1st repeat and so on)
)

display(CDNOW_master.head().collect())

CustID,Date,Quant,Spend,Spend Scaled,PurchDay,DoR
i32,date,i64,f64,i64,u16,u16
1,1997-01-01,1,11.77,1177,1,0
2,1997-01-12,6,89.0,8900,12,0
3,1997-01-02,2,20.76,2076,2,0
3,1997-03-30,2,20.76,2076,89,1
3,1997-04-02,2,19.54,1954,92,2


In [3]:
RptSpend = (
    CDNOW_master
    .filter(pl.col('PurchDay') <= 273)
    .with_columns(pl.when(pl.col('DoR') > 0)
                  .then(pl.col('Spend Scaled'))
                  .otherwise(0)
                  .alias('Repeat Spend (Scaled)'))
    .with_columns(pl.col('PurchDay').filter(pl.col('DoR') == 0)
                  .first()
                  .over('CustID')
                  .alias('Trial Day'))
    .group_by('CustID')
    .agg(pl.col('Repeat Spend (Scaled)').sum(), pl.col('Trial Day').max())
    .sort('CustID')
)

# Sampling technique - Python Method:
# id_df = (
#     RptSpend.collect()
#     .with_columns(((pl.col('Trial Day') - 1) // 7 + 1).alias('Trial Week'))
#     .sort(['Trial Week','Repeat Spend (Scaled)', 'CustID'], descending=[False, True, False], maintain_order=True)
# )

# sampledID = id_df[9::10].select('CustID')

In [4]:
# MATLAB Sampling (due to numerical float precision handling differences, original sampling results cannot be replicated unless spend is scaled in MATLAB)
CDNOW_sample = (
    pl.scan_csv(source='data/CDNOW/CDNOW_sample.csv',
                has_header=False,
                separator=',',
                schema={'CustID': pl.Int32,
                        'NewID': pl.Int32,
                        'Date': pl.String,
                        'Quant': pl.Int16,
                        'Spend': pl.Float64})
    .with_columns(pl.col('Date').str.to_date("%Y%m%d"))
    .with_columns((pl.col('Date') - pl.date(1996,12,31)).dt.total_days().cast(pl.UInt16).alias('PurchDay'))
    .with_columns((pl.col('Spend')*100).round(0).cast(pl.Int64).alias('Spend Scaled'))
    .group_by('CustID', 'Date')
    .agg(pl.col('*').exclude('PurchDay').sum(), pl.col('PurchDay').max())
    .sort('CustID', 'Date')
    .with_columns((pl.col("CustID").cum_count().over("CustID") - 1).cast(pl.UInt16).alias("DoR"))      
)

Creating summaries of the 1/10th sample data. We divide the 78 weeks in half: Period 1 is a 39-week calibration period while Period 2 is a 39-week longitudinal holdout used for model validation.

In [5]:
calwk = 273 # 39 week calibration period

# The number of repeat transactions made by each customer in each period
px = (
    CDNOW_sample
    .group_by('CustID', maintain_order=True)
    .agg(
        pl.col('PurchDay')
        .filter((pl.col('PurchDay') <= calwk) & (pl.col('DoR') > 0))
        .count()
        .alias('P1X'), # Period 1: Calibration Period

        pl.col('PurchDay')
        .filter((pl.col('PurchDay') > calwk) & (pl.col('DoR') > 0))
        .count()
        .alias('P2X')  # Period 2: Longitudinal Holdout Period      
    )
)

# The number of CDs purchased and total spend across these repeat transactions
pSpendQuant = (
    CDNOW_sample
    .join(px, on='CustID', how='left')
    .group_by('CustID', maintain_order=True)
    .agg(
        
        pl.col('Spend Scaled')
        .filter((pl.col('DoR') > 0) & (pl.col('DoR') <= pl.col('P1X')) & (pl.col('P1X') != 0))
        .sum()
        .alias('P1X Spend'),
        
        pl.col('Quant')
        .filter((pl.col('DoR') > 0) & (pl.col('DoR') <= pl.col('P1X')) & (pl.col('P1X') != 0))
        .sum()
        .alias('P1X Quant'),        
        
        pl.col('Spend Scaled')
        .filter((pl.col('DoR') > 0) & (pl.col('DoR') > pl.col('P1X')))
        .sum()
        .alias('P2X Spend'),
        
        pl.col('Quant')
        .filter((pl.col('DoR') > 0) & (pl.col('DoR') > pl.col('P1X')))
        .sum()
        .alias('P2X Quant')                
    )
)

# The average spend per repeat transaction
mx = (
    pSpendQuant
    .join(px, on='CustID', how='left')
    .with_columns(
        (pl.col('P1X Spend') / pl.col('P1X')).alias('Avg Spend per Repeat')
    ).fill_nan(0)
)

When fitting models such as the Pareto/NBD and BG/NBD to these data, we also want to know the “recency” information for each customer, as well as their effective calibration period:

In [6]:
# time of last calibration period repeat purchase (in weeks)
ttlrp = (
    CDNOW_sample
    .join(px, on='CustID', how='left')
    .with_columns(
        pl.col('PurchDay').filter(pl.col('DoR') == 0)
        .first()
        .over('CustID')
        .alias('Trial Day')
    )
    .group_by('CustID', maintain_order=True)
    .agg(
        pl.col('PurchDay', 'Trial Day')
        .filter(pl.col('DoR') <= pl.col('P1X'))
        .max()
        # .alias('LastPurch')
    )
    .with_columns(
        # effective calibration period (in weeks)
        ((pl.col('PurchDay') - pl.col('Trial Day')) / 7).alias('Time to Last Repeat'),
        ((calwk - pl.col('Trial Day'))/7).alias('Eff. Calibration Period')
    )
    .drop('PurchDay', 'Trial Day')
)

Creating Summaries

In [7]:
# What is the total number of CDs purchased each week?
weeklysales = (
    CDNOW_master
    .with_columns(((pl.col('PurchDay') - 1) // 7 + 1).alias('Week'))
    .group_by('Week')
    .agg(pl.col('Quant').sum())
    .sort('Week')
)

(
    ChartTemp(weeklysales.collect())
    .line_encode(y_col='Quant', y_title='Units Purchased', x_range=78)
    .line_prop('Weekly Repeat Sales')
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [8]:
# How many people made their first-ever (“trial”) purchase each week?
weeklytriers = (
    CDNOW_master
    .filter(pl.col('DoR') == 0)
    .with_columns(((pl.col('PurchDay') - 1) // 7 + 1).alias('Week'))
    .group_by('Week')
    .agg(pl.len().alias('Incremental Triers'))
    .sort('Week')
    
)
weeklytriers.collect()

Week,Incremental Triers
u16,u32
1,1574
2,1642
3,1822
4,1924
5,2164
…,…
8,2034
9,2198
10,2165


In [9]:
# What is the total number of CDs purchased by triers in their trial week?
weeklytrierquant = (
    CDNOW_master
    .with_columns(((pl.col('PurchDay') - 1) // 7 + 1).alias('Week'))
    .with_columns(pl.when(pl.col('DoR') == 0).then(pl.col('Week')).fill_null(strategy='forward').over('CustID').alias('Trial Week'))  
    .filter(pl.col('Trial Week') == pl.col('Week'))  # Any repeat purchasing by a customer in their trial week is added to their trial purchase
    .group_by('Week')
    .agg(pl.col('Quant').sum().alias('Triers Quant'))
    .sort('Week')
)

weeklytrierquant.collect()

Week,Triers Quant
u16,i64
1,3627
2,3630
3,4015
4,4347
5,4867
…,…
8,4501
9,4896
10,4795


In [10]:
weeklysalessplit = (
    weeklysales
    .join(weeklytrierquant, on='Week', how='left')
    .fill_null(0)
    .with_columns(
        (pl.col('Quant') - pl.col('Triers Quant')).alias('Repeat Quant')
    )
    .rename({'Quant': 'Total', 'Triers Quant': 'Trial', 'Repeat Quant': 'Repeat'})
)

In [11]:
# What is the distribution of the number of units purchased in each of the first 12 weeks?
dist_table = (
    CDNOW_master
    .with_columns(((pl.col('PurchDay') - 1) // 7 + 1).alias('Week'))
    .filter(pl.col('Week') <= 12)
    .group_by('Week', 'CustID')
    .agg(pl.col('Quant').sum())    
    .sort('Week').collect()
    .pivot(on='Week', index='Quant', values='Quant', aggregate_function='len')
    .sort('Quant')
    .fill_null(0)
    .to_numpy().astype(np.int32)
)
TableOne = np.zeros((11,13), dtype=np.int32)
TableOne[1:11,0] = dist_table[0:10,0]
TableOne[1:10,1:] = dist_table[0:9,1:]
TableOne[10,1:] = np.sum(dist_table[9:,1:], axis=0)
TableOne[0,1:] = weeklytriers.collect()['Incremental Triers'].cum_sum() - np.sum(TableOne[1:,1:], axis=0)
TableOne_df = pl.from_numpy(TableOne, schema=[str(i) for i in range(13)], orient='row').with_columns(pl.col('0').cast(pl.String))

(
    GT(TableOne_df, rowname_col='0')
    .tab_header(title="Distribution of the Number of CDNOW Units Purchased", subtitle='First 12 Weeks')
    .tab_stubhead('Units Purchased')
    .fmt_integer()
    .tab_spanner(label='Week', columns=[str(i) for i in range(1, 13)])
    .opt_stylize()
)

GT(_tbl_data=shape: (11, 13)
┌─────┬─────┬──────┬──────┬───┬───────┬───────┬───────┬───────┐
│ 0   ┆ 1   ┆ 2    ┆ 3    ┆ … ┆ 9     ┆ 10    ┆ 11    ┆ 12    │
│ --- ┆ --- ┆ ---  ┆ ---  ┆   ┆ ---   ┆ ---   ┆ ---   ┆ ---   │
│ str ┆ i32 ┆ i32  ┆ i32  ┆   ┆ i32   ┆ i32   ┆ i32   ┆ i32   │
╞═════╪═════╪══════╪══════╪═══╪═══════╪═══════╪═══════╪═══════╡
│ 0   ┆ 0   ┆ 1478 ┆ 3033 ┆ … ┆ 14698 ┆ 16774 ┆ 18881 ┆ 20902 │
│ 1   ┆ 750 ┆ 852  ┆ 984  ┆ … ┆ 1397  ┆ 1444  ┆ 1387  ┆ 1148  │
│ 2   ┆ 383 ┆ 387  ┆ 456  ┆ … ┆ 644   ┆ 659   ┆ 677   ┆ 663   │
│ 3   ┆ 191 ┆ 214  ┆ 270  ┆ … ┆ 365   ┆ 374   ┆ 355   ┆ 367   │
│ 4   ┆ 95  ┆ 120  ┆ 114  ┆ … ┆ 179   ┆ 187   ┆ 199   ┆ 182   │
│ …   ┆ …   ┆ …    ┆ …    ┆ … ┆ …     ┆ …     ┆ …     ┆ …     │
│ 6   ┆ 36  ┆ 40   ┆ 42   ┆ … ┆ 75    ┆ 71    ┆ 72    ┆ 54    │
│ 7   ┆ 18  ┆ 12   ┆ 27   ┆ … ┆ 41    ┆ 37    ┆ 30    ┆ 43    │
│ 8   ┆ 12  ┆ 15   ┆ 9    ┆ … ┆ 23    ┆ 29    ┆ 24    ┆ 32    │
│ 9   ┆ 9   ┆ 9    ┆ 8    ┆ … ┆ 14    ┆ 9     ┆ 12    ┆ 16    │
│ 10  ┆ 25  ┆ 17   ┆ 27   ┆ … ┆ 48    ┆ 42    ┆ 50    ┆ 43    │
└─────┴─────┴──────┴──────┴───┴───────┴───────┴───────┴───────┘, _body=<great_tables._gt_data.Body object at 0x00000242991F1070>, _boxhead=Boxhead([ColInfo(var='0', type=<ColInfoTypeEnum.stub: 2>, column_label='0', column_align='left', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='1', column_align='right', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='2', column_align='right', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='3', column_align='right', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='4', column_align='right', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='5', column_align='right', column_width=None), ColInfo(var='6', type=<ColInfoTypeEnum.default: 1>, column_label='6', column_align='right', column_width=None), ColInfo(var='7', type=<ColInfoTypeEnum.default: 1>, column_label='7', column_align='right', column_width=None), ColInfo(var='8', type=<ColInfoTypeEnum.default: 1>, column_label='8', column_align='right', column_width=None), ColInfo(var='9', type=<ColInfoTypeEnum.default: 1>, column_label='9', column_align='right', column_width=None), ColInfo(var='10', type=<ColInfoTypeEnum.default: 1>, column_label='10', column_align='right', column_width=None), ColInfo(var='11', type=<ColInfoTypeEnum.default: 1>, column_label='11', column_align='right', column_width=None), ColInfo(var='12', type=<ColInfoTypeEnum.default: 1>, column_label='12', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002429AA2BCB0>, _spanners=Spanners([SpannerInfo(spanner_id='Week', spanner_level=0, spanner_label='Week', spanner_units=None, spanner_pattern=None, vars=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'], built=None)]), _heading=Heading(title='Distribution of the Number of CDNOW Units Purchased', subtitle='First 12 Weeks', preheader=None), _stubhead='Units Purchased', _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000002429A986840>, _formats=[<great_tables._gt_data.FormatInfo object at 0x000002429AEE3050>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), 

In [12]:
dist_table = (
    CDNOW_master
    .with_columns(((pl.col('PurchDay') - 1) // 7 + 1).alias('Week'))  
    .filter(pl.col('Week') <= 12)  
    .group_by('CustID', 'Week')
    .agg(pl.col('Quant').cast(pl.Int32).sum()) # Sum quantity purchased by each customer within each week 
    .group_by('Week', 'Quant')
    .agg(pl.col('CustID').len().alias('Count'))
    .sort('Quant')
    .collect()
)

dist_table_10_plus = (
    dist_table.filter(pl.col('Quant') >= 10)
    .group_by('Week')
    .agg(pl.col('Count').sum())
    .sort('Week')
    .with_columns(pl.lit(10).alias('Quant'))
    .select('Week', 'Quant', 'Count')
)

dist_table_1 = dist_table.filter(pl.col('Quant') < 10).vstack(dist_table_10_plus)

cumweeklytriers = (
    weeklytriers.collect()
    .with_columns(pl.col('Incremental Triers').cum_sum())
    .join(dist_table_1.group_by('Week').agg(pl.col('Count').sum()), on='Week', how='left')
    .with_columns((pl.col('Incremental Triers') - pl.col('Count')).alias('Count'))
    .with_columns(pl.lit(0).alias('Quant')).select('Week', 'Quant', 'Count')
)

TableOne = (
    dist_table_1.vstack(cumweeklytriers).sort('Week', 'Quant')
    .pivot(on='Week', index='Quant', values='Count')
)

(
    GT(TableOne, rowname_col='Quant')
    .tab_header(title="Distribution of the Number of CDNOW Units Purchased", subtitle='First 12 Weeks')
    .tab_stubhead('Units Purchased')
    .fmt_integer()
    .tab_spanner(label='Week', columns=[str(i) for i in range(1, 13)])
    .opt_stylize()
)

GT(_tbl_data=shape: (11, 13)
┌───────┬─────┬──────┬──────┬───┬───────┬───────┬───────┬───────┐
│ Quant ┆ 1   ┆ 2    ┆ 3    ┆ … ┆ 9     ┆ 10    ┆ 11    ┆ 12    │
│ ---   ┆ --- ┆ ---  ┆ ---  ┆   ┆ ---   ┆ ---   ┆ ---   ┆ ---   │
│ i32   ┆ u32 ┆ u32  ┆ u32  ┆   ┆ u32   ┆ u32   ┆ u32   ┆ u32   │
╞═══════╪═════╪══════╪══════╪═══╪═══════╪═══════╪═══════╪═══════╡
│ 0     ┆ 0   ┆ 1478 ┆ 3033 ┆ … ┆ 14698 ┆ 16774 ┆ 18881 ┆ 20902 │
│ 1     ┆ 750 ┆ 852  ┆ 984  ┆ … ┆ 1397  ┆ 1444  ┆ 1387  ┆ 1148  │
│ 2     ┆ 383 ┆ 387  ┆ 456  ┆ … ┆ 644   ┆ 659   ┆ 677   ┆ 663   │
│ 3     ┆ 191 ┆ 214  ┆ 270  ┆ … ┆ 365   ┆ 374   ┆ 355   ┆ 367   │
│ 4     ┆ 95  ┆ 120  ┆ 114  ┆ … ┆ 179   ┆ 187   ┆ 199   ┆ 182   │
│ …     ┆ …   ┆ …    ┆ …    ┆ … ┆ …     ┆ …     ┆ …     ┆ …     │
│ 6     ┆ 36  ┆ 40   ┆ 42   ┆ … ┆ 75    ┆ 71    ┆ 72    ┆ 54    │
│ 7     ┆ 18  ┆ 12   ┆ 27   ┆ … ┆ 41    ┆ 37    ┆ 30    ┆ 43    │
│ 8     ┆ 12  ┆ 15   ┆ 9    ┆ … ┆ 23    ┆ 29    ┆ 24    ┆ 32    │
│ 9     ┆ 9   ┆ 9    ┆ 8    ┆ … ┆ 14    ┆ 9     ┆ 12    ┆ 16    │
│ 10    ┆ 25  ┆ 17   ┆ 27   ┆ … ┆ 48    ┆ 42    ┆ 50    ┆ 43    │
└───────┴─────┴──────┴──────┴───┴───────┴───────┴───────┴───────┘, _body=<great_tables._gt_data.Body object at 0x000002429AF63050>, _boxhead=Boxhead([ColInfo(var='Quant', type=<ColInfoTypeEnum.stub: 2>, column_label='Quant', column_align='right', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='1', column_align='center', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='2', column_align='center', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='3', column_align='center', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='4', column_align='center', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='5', column_align='center', column_width=None), ColInfo(var='6', type=<ColInfoTypeEnum.default: 1>, column_label='6', column_align='center', column_width=None), ColInfo(var='7', type=<ColInfoTypeEnum.default: 1>, column_label='7', column_align='center', column_width=None), ColInfo(var='8', type=<ColInfoTypeEnum.default: 1>, column_label='8', column_align='center', column_width=None), ColInfo(var='9', type=<ColInfoTypeEnum.default: 1>, column_label='9', column_align='center', column_width=None), ColInfo(var='10', type=<ColInfoTypeEnum.default: 1>, column_label='10', column_align='center', column_width=None), ColInfo(var='11', type=<ColInfoTypeEnum.default: 1>, column_label='11', column_align='center', column_width=None), ColInfo(var='12', type=<ColInfoTypeEnum.default: 1>, column_label='12', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002429AF62F90>, _spanners=Spanners([SpannerInfo(spanner_id='Week', spanner_level=0, spanner_label='Week', spanner_units=None, spanner_pattern=None, vars=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'], built=None)]), _heading=Heading(title='Distribution of the Number of CDNOW Units Purchased', subtitle='First 12 Weeks', preheader=None), _stubhead='Units Purchased', _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000002429AF62060>, _formats=[<great_tables._gt_data.FormatInfo object at 0x000002429AF62C30>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(s

Week 1 Trial Model

As a refresher (or primer) on estimating the parameters of a basic probability model using SciPy, let us consider fitting the trial submodel to the week 1 data. The column of data in `TableOne` corresponding to week 1 presents *trial-week-only* purchases by a group of 1574 customers. Our goal is to fit the following shifted beta-geometric model to these data:

$$
\begin{aligned}
P(T_{w} = x) &= \int^{1}_{0}P(T_{w} = x \mid q_{T})g(q_{T})dq_{T} \\
             &=\begin{cases}
               \frac{B(\alpha_{T} + 1,\beta_{T} + x - 1)}{B(\alpha_{T}+1,\beta_{T})} &\quad x=1,2,\ldots \\
                0 &\quad x=0 \\
                \end{cases}
\end{aligned}
$$

The shifted beta-geometric probabilities can be computed by recursion using the expressions:

$$
\begin{aligned}
P(T_{w} = x) &= \begin{cases}
                    0 &\quad x=0 \\
                    \frac{\alpha_{T}}{\alpha_{T} + \beta_{T}} &\quad x=1\\
                    \frac{\beta_{T} + x - 2}{\alpha_{T} + \beta_{T} + x -1} P(T_{w}=x-1) &\quad x\ge 2 \\
                \end{cases}
\end{aligned}
$$

The probability of making 10+ purchases in a trial week is simply:

$$1-\sum^{9}_{x=1}P(T_{1}=x)$$

The shifted beta-geometric model has two parameters, $\alpha_{T}$ and $\beta_{T}$ . Maximum likelihood estimates of these two model parameters are found by maximizing the following log-likelihood function: 

$$LL=\sum^{9}_{x=1}n_{1x}\ln\left[P(T_{1}=x)\right]+\left(n_{1}-\sum^{9}_{x=1}n_{1x}\right)\ln\left[1-\sum^{9}_{x=1}P(T_{1}=x)\right]$$

Where $n_{1}$ is the total number of triers in week 1 (1574) and $n_{1x}$ is the number of people making $x$ purchases in week 1.

In [13]:
def trial_parameters(n1x, guess=[1,1]):
    
    def log_likelihood(x):
        alpha_T, beta_T = x[0], x[1]
        
        # Shifted beta-geometric probabilities of making x purchases
        p_t_1x = np.zeros(n1x.shape)
        p_t_1x[0], p_t_1x[1] = 0, alpha_T / (alpha_T + beta_T)
        x = np.arange(2, len(n1x)-1)
        p_t_1x[2:len(n1x)-1] = (beta_T + x - 2) / (alpha_T + beta_T + x - 1)
        p_t_1x[1:len(n1x)-1] = np.cumprod(p_t_1x[1:len(n1x)-1])
        p_t_1x[-1] = 1 - np.sum(p_t_1x[:-1])     
          
        return -np.sum(np.log(p_t_1x, where=p_t_1x != 0) * n1x) 
    
    return minimize(log_likelihood, guess, bounds=[(0, np.inf), (0, np.inf)])

In [14]:
n1x = TableOne['1'].to_numpy()

result = trial_parameters(n1x)
alpha_T, beta_T, ll = result.x[0], result.x[1], result.fun

w1_alpha_T, w1_beta_T = alpha_T, beta_T

display_markdown(f'''$\\alpha_{{T}}$ = {alpha_T:0.4f}

$\\beta_{{T}}$ = {beta_T:0.4f}

Log-Likelihood = {-ll:0.4f}''', raw=True)

$\alpha_{T}$ = 5.9117

$\beta_{T}$ = 6.2832

Log-Likelihood = -2378.0156

Chi-Squared Goodness of Fit Test

We first have to compute the expected number of people buying $1, 2, \ldots , 9, 10+$ units in their trial week. We have:

$$E(n_{1x})=1574\times P(T_{1}=x)$$

The chi-squared goodness of fit test statistic is computed as:

$$\chi^{2}=\sum^{10+}_{x=1}\frac{\left[n_{1x}-E(n_{1x})\right]^{2}}{E(n_{1x})}$$

In [15]:
p_t_1x = np.zeros(n1x.shape)
p_t_1x[0], p_t_1x[1] = 0, alpha_T / (alpha_T + beta_T)
x = np.arange(2, len(n1x)-1)
p_t_1x[2:len(n1x)-1] = (beta_T + x - 2) / (alpha_T + beta_T + x - 1)
p_t_1x[1:len(n1x)-1] = np.cumprod(p_t_1x[1:len(n1x)-1])
p_t_1x[-1] = 1 - np.sum(p_t_1x[:-1])     

n1 = weeklytriers.filter(pl.col('Week') == 1).select('Incremental Triers').collect().item(0,0) 
E_n1x = p_t_1x * n1
chi_square = np.sum(np.divide((n1x - E_n1x)**2, E_n1x, where=E_n1x != 0))

df = 7
critical_value = chi2.ppf(0.95, df=df)

display_markdown(f'''$E_{{n_{{1x}}}}$ = {np.round(E_n1x[1:],1)}

$\\chi^{{2}}$ = {chi_square:0.4f}

Degrees of freedom = {df}

Critical Value = {critical_value:0.4f}''', raw=True)

$E_{n_{1x}}$ = [763.  363.3 186.4 101.6  58.3  34.8  21.6  13.8   9.1  22. ]

$\chi^{2}$ = 3.3117

Degrees of freedom = 7

Critical Value = 14.0671

As the value of the sample test statistic is less than the critical value, we conclude that the shifted beta-geometric distribution adequately fits the data.

In [16]:
w1_trial_df = pl.DataFrame({'Number of Units Purchased':np.arange(1,11), 'Observed': n1x[1:], 'Expected': E_n1x[1:]})
w1_trial_df = w1_trial_df.unpivot(on=['Observed', 'Expected'], index='Number of Units Purchased', variable_name='Legend' , value_name='Frequency')

ChartTemp(w1_trial_df).mark_bar().encode(
    x=alt.X('Number of Units Purchased:O', axis=alt.Axis(labelAngle=0)),
    y='Frequency:Q',
    color='Legend:N',
    xOffset='Legend:N'
).line_prop('Fit of Week 1 Trial Model')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


Calibrating the Full Model

We now turn our attention to the task of estimating the parameters of the full model. Our goal is to construct the following log-likelihood function:

$$
\begin{align*}
LL= & \sum^{9}_{x=1}n_{1x}\ln\left[P(T_{1}=x)\right]+\left(n_{1}-\sum^{9}_{x=1}n_{1x}\right)\ln\left[1-\sum^{9}_{x=1}P(T_{1}=x)\right] + \\
& \sum^{12}_{w=2} \Biggl\{\sum^{9}_{x=0}n_{wx} \ln[P(X_{w}=x)] + \left(n_{w} - \sum^{9}_{x=0}n_{wx}\right) \ln\left[1-\sum^{9}_{x=0}P(X_{w}=x)\right] \Biggr\}
\end{align*}
$$

where $n_{wx}$ is the number of people making $x$ purchases in week $w$.

At the heart of this log-likelihood function is $P(X_{w} = x)$, the probability that an eligible customer purchases $x$ units in week $w$, as given in the equation:

$$P(X_{w} = x)=\frac{1}{\sum^{w}_{i=1}n_{i}} \Biggl[n_{w}P(T_{w}=x) + \sum^{w-1}_{i=1}n_{i}P(R_{w\mid i}=x)\Biggr]$$

The aggregate distribution of the number of units purchased in week $w$ by a week $i$ trialist $(w \gt i)$ is given by:

$$
\begin{align*}
P(R_{w \mid i} = x) &= \delta_{x=0} \pi_{w \mid i} + (1 - \pi_{w \mid i}) \int_0^1 P(R_w = x \mid q_R) g(q_R) \, dq_R \\
&= \delta_{x=0} \pi_{w \mid i} + (1 - \pi_{w \mid i}) \frac{B(\alpha_R + 1, \beta_R + x)}{B(\alpha_R, \beta_R)},
\end{align*}
$$

where $\delta_{x=0}$, the Kronecker delta, equals $1$ if $x = 0$, and $0$ otherwise. We call this the "time-dependent, zero-inflated beta-geometric" distribution.

Probabilities associated with the "time dependent, zero-inflated beta-geometric" distribution can be computed using the following forward recursive relationship:

$$
P(R_{w \mid i} = x) =
\begin{cases}
1 - \gamma(w - i)^\delta \left( \frac{\beta_R}{\alpha_R + \beta_R} \right), & x = 0 \\[10pt]
\gamma(w - i)^\delta \frac{\alpha_R \beta_R}{(\alpha_R + \beta_R)(\alpha_R + \beta_R + 1)}, & x = 1 \\[10pt]
\frac{\beta_R + x - 1}{\alpha_R + \beta_R + x} P(R_{w \mid i} = x - 1), & x \geq 2
\end{cases}
$$

However, rather than directly use the recursive relationship given in the equation above, we take the following apporach. We first compute the probability that a week 1 trier is a "possible repeat buyer" in weeks 2-12 using $\gamma(w-1)^{\delta}$.

Next we need to create the expressions for the beta-geometric probabilities of making $x$ purchases $(x = 0, 1, \ldots , 9, 10+)$, given $\alpha_R$ and $\beta_R$, for someone who is a “possible repeat buyer”. The beta-geometric probabilities can be computed using the following recursive relationship:

$$
P(R = x \mid \text{possible repeat buyer}) =
\begin{cases}
\frac{\alpha_R}{\alpha_R + \beta_R}, & x = 0 \\[10pt]
\frac{\beta_R + x - 1}{\alpha_R + \beta_R + x} P(R = x - 1), & x \ge 1 \\[10pt]
\end{cases}
$$

The probability of a "possible repeat buyer" making 10+ purchases is simply $1 - \sum^{9}_{x=0} P(R=x)$.

As the sample data are in the form of a table documenting the number of people purchasing $0, 1, \ldots , 9, 10+$ units (CDs) for each of the 12 weeks, we need to create a table that gives us $P(X_w = x), x = 0, 1, \ldots , 9, 10+$ and $w = 1, 2, \ldots 12$, given values of the six model parameters $(\alpha_T, \beta_T, \alpha_R, \beta_R, \gamma, \delta)$.

We see that $P(X_w = x)$ is simply a weighted average of the week-of-trial-specific probabilities of purchasing $x$ units in week $w$. As an intermediate step, we build twelve tables that give us the probability of purchasing $x$ units in week $w$, one for each trial week. These will then be aggregated and the log-likelihood created.

In [17]:
def bg_parameters(distMAT, weeks, units_range, incr_triers, guesses=[0.01,0.01,0.01,0.01,0.01,0]):
    
    def log_likelihood(x):
        alpha_T, beta_T, alpha_R, beta_R, gamma, delta = x
        
        # Weekly P(possible repeat buyer)
        weekly_p_repeat = gamma * (weeks[1:] - 1)**delta
        
        # Distribution of repeat purchase volume (given possible repeat buyer)
        p_dist_repeat = np.zeros(units_range.shape)
        p_dist_repeat[0] = alpha_R / (alpha_R + beta_R)
        p_dist_repeat[1:-1] = (beta_R + units_range[1:-1] - 1) / (alpha_R + beta_R + units_range[1:-1])
        p_dist_repeat = np.cumprod(p_dist_repeat)
        p_dist_repeat[-1] = 1 - np.sum(p_dist_repeat[:-1])

        # Week n Triers
        w_n_triers = p_dist_repeat.reshape(-1,1) * np.append([0], weekly_p_repeat)
        w_n_triers[0,1:] =  1 -  weekly_p_repeat + weekly_p_repeat * p_dist_repeat[0]
        w_n_triers[1,0] = alpha_T / (alpha_T + beta_T)
        w_n_triers[2:-1,0] = (beta_T + distMAT[2:-1,0] - 2) / (alpha_T + beta_T + distMAT[2:-1,0] - 1)
        w_n_triers[1:,0] = np.cumprod(w_n_triers[1:,0])
        w_n_triers[-1,0] = 1 - np.sum(w_n_triers[:-1,0])        
        
        cum_triers = np.cumsum(incr_triers)

        # P(X_{w} = x) - Weighted Probabilities
        weight_probs = np.zeros_like(w_n_triers)
        for i in range(w_n_triers.shape[0]):
            for j in range(w_n_triers.shape[1]):
                weight_probs[i,j] = np.sum(incr_triers[:j+1] * w_n_triers[i,j::-1]) / cum_triers[j]        
        
        # Log-Likelihood Computation
        ll_components = distMAT[:,1:] * np.log(weight_probs, where=weight_probs > 0)
        ll = np.sum(ll_components)        
        
        return -ll
    
    bnds = [(1e-5, np.inf) for _ in range(len(guesses)-1)] + [(-np.inf, np.inf)]
    return minimize(log_likelihood, guesses, bounds=bnds)   

In [18]:
distMAT = TableOne.to_numpy()
trial_weeks = np.array(TableOne.columns[1:], dtype=np.int32)
units_range = np.arange(distMAT.shape[0])
incr_triers = weeklytriers.select('Incremental Triers').collect().to_numpy().flatten()

cum_triers = np.cumsum(incr_triers)

result = bg_parameters(distMAT, trial_weeks, units_range, incr_triers)

alpha_T, beta_T, alpha_R, beta_R, gamma, delta = result.x
ll = result.fun

allW_alpha_T, allW_beta_T = alpha_T, beta_T

display_markdown(f'''$\\alpha_{{T}}$ = {alpha_T:0.4f}

$\\beta_{{T}}$ = {beta_T:0.4f}

$\\alpha_{{R}}$ = {alpha_R:0.4f}

$\\beta_{{R}}$ = {beta_R:0.4f}

$\\gamma$ = {gamma:0.4f}

$\\delta$ = {delta:0.4f}

Log-Likelihood = {-ll:0.6f}''', raw=True)

$\alpha_{T}$ = 6.9066

$\beta_{T}$ = 7.1924

$\alpha_{R}$ = 5.0134

$\beta_{R}$ = 5.5804

$\gamma$ = 0.1216

$\delta$ = -0.2915

Log-Likelihood = -112923.922872

In [19]:
# Weekly P(possible repeat buyer)
weekly_p_repeat = gamma * (trial_weeks[1:] - 1)**delta

# Distribution of repeat purchase volume (given possible repeat buyer)
p_dist_repeat = np.zeros(units_range.shape)
p_dist_repeat[0] = alpha_R / (alpha_R + beta_R)
p_dist_repeat[1:-1] = (beta_R + units_range[1:-1] - 1) / (alpha_R + beta_R + units_range[1:-1])
p_dist_repeat = np.cumprod(p_dist_repeat)
p_dist_repeat[-1] = 1 - np.sum(p_dist_repeat[:-1])

# Week n Triers
w_n_triers = p_dist_repeat.reshape(-1,1) * np.append([0], weekly_p_repeat)
w_n_triers[0,1:] =  1 -  weekly_p_repeat + weekly_p_repeat * p_dist_repeat[0]
w_n_triers[1,0] = alpha_T / (alpha_T + beta_T)
w_n_triers[2:-1,0] = (beta_T + distMAT[2:-1,0] - 2) / (alpha_T + beta_T + distMAT[2:-1,0] - 1)
w_n_triers[1:,0] = np.cumprod(w_n_triers[1:,0])
w_n_triers[-1,0] = 1 - np.sum(w_n_triers[:-1,0])

# P(X_{w} = x) - Weighted Probabilities
weight_probs = np.zeros_like(w_n_triers)
for i in range(w_n_triers.shape[0]):
    for j in range(w_n_triers.shape[1]):
        weight_probs[i,j] = np.sum(incr_triers[:j+1] * w_n_triers[i,j::-1]) / cum_triers[j]

# Log-Likelihood Computation
ll_components = distMAT[:,1:] * np.log(weight_probs, where=weight_probs != 0)
ll = np.sum(ll_components)

# Expected Distribution of Sales
e_distMAT = weight_probs * cum_triers

# chi-squared goodness of fit test statistic
chi_square = np.sum(np.divide((distMAT[:,1:] - e_distMAT)**2, e_distMAT, where=distMAT[:,1:]!=0))
df = 113
critical_value = chi2.ppf(0.95, df=df)

In [20]:
e_distTable = (
    pl.DataFrame({'Quant': np.arange(11, dtype=np.int32)})
    .hstack(pl.from_numpy(e_distMAT.astype(np.int32), schema=TableOne.columns[1:]))
)

(
    GT(e_distTable, rowname_col='Quant')
    .tab_header(title="Estimated Distribution of the Number of CDNOW Units Purchased", subtitle='First 12 Weeks')
    .tab_stubhead('Units Purchased')
    .fmt_integer()
    .tab_spanner(label='Week', columns=[str(i) for i in range(1, 13)])
    .opt_stylize()
)

GT(_tbl_data=shape: (11, 13)
┌───────┬─────┬──────┬──────┬───┬───────┬───────┬───────┬───────┐
│ Quant ┆ 1   ┆ 2    ┆ 3    ┆ … ┆ 9     ┆ 10    ┆ 11    ┆ 12    │
│ ---   ┆ --- ┆ ---  ┆ ---  ┆   ┆ ---   ┆ ---   ┆ ---   ┆ ---   │
│ i32   ┆ i32 ┆ i32  ┆ i32  ┆   ┆ i32   ┆ i32   ┆ i32   ┆ i32   │
╞═══════╪═════╪══════╪══════╪═══╪═══════╪═══════╪═══════╪═══════╡
│ 0     ┆ 0   ┆ 1473 ┆ 3028 ┆ … ┆ 14689 ┆ 16807 ┆ 18898 ┆ 20872 │
│ 1     ┆ 771 ┆ 847  ┆ 973  ┆ … ┆ 1375  ┆ 1394  ┆ 1363  ┆ 1269  │
│ 2     ┆ 367 ┆ 405  ┆ 467  ┆ … ┆ 669   ┆ 679   ┆ 666   ┆ 622   │
│ 3     ┆ 186 ┆ 207  ┆ 239  ┆ … ┆ 348   ┆ 354   ┆ 348   ┆ 326   │
│ 4     ┆ 100 ┆ 112  ┆ 130  ┆ … ┆ 191   ┆ 195   ┆ 192   ┆ 181   │
│ …     ┆ …   ┆ …    ┆ …    ┆ … ┆ …     ┆ …     ┆ …     ┆ …     │
│ 6     ┆ 33  ┆ 37   ┆ 43   ┆ … ┆ 66    ┆ 68    ┆ 67    ┆ 64    │
│ 7     ┆ 20  ┆ 22   ┆ 26   ┆ … ┆ 41    ┆ 42    ┆ 42    ┆ 40    │
│ 8     ┆ 12  ┆ 14   ┆ 16   ┆ … ┆ 26    ┆ 27    ┆ 27    ┆ 26    │
│ 9     ┆ 8   ┆ 9    ┆ 11   ┆ … ┆ 17    ┆ 18    ┆ 18    ┆ 17    │
│ 10    ┆ 17  ┆ 21   ┆ 25   ┆ … ┆ 42    ┆ 44    ┆ 45    ┆ 43    │
└───────┴─────┴──────┴──────┴───┴───────┴───────┴───────┴───────┘, _body=<great_tables._gt_data.Body object at 0x000002429AFABDD0>, _boxhead=Boxhead([ColInfo(var='Quant', type=<ColInfoTypeEnum.stub: 2>, column_label='Quant', column_align='right', column_width=None), ColInfo(var='1', type=<ColInfoTypeEnum.default: 1>, column_label='1', column_align='right', column_width=None), ColInfo(var='2', type=<ColInfoTypeEnum.default: 1>, column_label='2', column_align='right', column_width=None), ColInfo(var='3', type=<ColInfoTypeEnum.default: 1>, column_label='3', column_align='right', column_width=None), ColInfo(var='4', type=<ColInfoTypeEnum.default: 1>, column_label='4', column_align='right', column_width=None), ColInfo(var='5', type=<ColInfoTypeEnum.default: 1>, column_label='5', column_align='right', column_width=None), ColInfo(var='6', type=<ColInfoTypeEnum.default: 1>, column_label='6', column_align='right', column_width=None), ColInfo(var='7', type=<ColInfoTypeEnum.default: 1>, column_label='7', column_align='right', column_width=None), ColInfo(var='8', type=<ColInfoTypeEnum.default: 1>, column_label='8', column_align='right', column_width=None), ColInfo(var='9', type=<ColInfoTypeEnum.default: 1>, column_label='9', column_align='right', column_width=None), ColInfo(var='10', type=<ColInfoTypeEnum.default: 1>, column_label='10', column_align='right', column_width=None), ColInfo(var='11', type=<ColInfoTypeEnum.default: 1>, column_label='11', column_align='right', column_width=None), ColInfo(var='12', type=<ColInfoTypeEnum.default: 1>, column_label='12', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002429AFA8A40>, _spanners=Spanners([SpannerInfo(spanner_id='Week', spanner_level=0, spanner_label='Week', spanner_units=None, spanner_pattern=None, vars=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'], built=None)]), _heading=Heading(title='Estimated Distribution of the Number of CDNOW Units Purchased', subtitle='First 12 Weeks', preheader=None), _stubhead='Units Purchased', _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000002429AFABEF0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x000002429AFAA810>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scs

In [21]:
display_markdown(f'''$\\chi^{{2}}$ = {chi_square:0.4f}

Degrees of freedom = {df}

Critical Value = {critical_value:0.4f}''', raw=True)

$\chi^{2}$ = 129.2064

Degrees of freedom = 113

Critical Value = 138.8114

Creating the Sales Forecast

$$
E(N_w) =
\begin{cases}
n_w E(T_w) + \sum_{i=1}^{w-1} n_i E(R_{w \mid i}) & w \leq 12 \\
\sum_{i=1}^{12} n_i E(R_{w\mid i}) & w > 12
\end{cases}
$$

$$E(T_w) = \frac{\alpha_T + \beta_T - 1}{\alpha_T - 1}$$

$$E(R_{w\mid i}) = \gamma (w - i)^\delta \frac{\beta_R}{\alpha_R - 1}$$

In [22]:
from scipy.linalg import toeplitz

full_weeks = np.arange(52) + 1

# expected number of units purchased in weeks 1–52 by any given week 1 trier 
week_1_triers = np.zeros(full_weeks.shape[0])
week_1_triers[0] = (alpha_T + beta_T - 1)/(alpha_T - 1) # expected number of units purchased in the trial week    
week_1_triers[1:] = gamma * (full_weeks[1:] - 1)**delta * beta_R / (alpha_R - 1) # expected repeat sales, number of units purchased w−i weeks after trial

# mean weekly unit purchases for any week i trier
# expected total number of units sold in any given week is
mean_weekly_unit_purchases = toeplitz(week_1_triers, r=np.zeros(trial_weeks.shape[0]))

# estimate of weekly unit sales - expected total number of units sold in weeks 1–52
e_weekly_sales = np.sum(incr_triers * mean_weekly_unit_purchases, axis=1)

In [23]:
# Without SciPy's Toeplitz Constructor
rows = full_weeks.shape[0]
cols = trial_weeks.shape[0]

mean_weekly_unit_purchases = np.zeros((rows, cols))

row_indices = np.arange(rows)[:, None]
col_indices = np.arange(cols)[None, :]
lag_mask = row_indices >= col_indices

mean_weekly_unit_purchases[lag_mask] = week_1_triers[(row_indices - col_indices)[lag_mask]]
e_weekly_sales = np.sum(incr_triers * mean_weekly_unit_purchases, axis=1)

In [24]:
e_total_sales = e_weekly_sales[:12]
e_trial_sales = mean_weekly_unit_purchases.diagonal() * incr_triers
e_repeat_sales = e_total_sales - e_trial_sales

e_weekly_sales_df = pl.DataFrame({'Week': trial_weeks, 
                               'Total Sales': e_total_sales, 
                               'Trial Sales': e_trial_sales, 
                               'Repeat Sales': e_repeat_sales})

e_weekly_sales_df = e_weekly_sales_df.unpivot(index='Week', on=['Total Sales', 'Trial Sales', 'Repeat Sales'], variable_name='Estimated Sales', value_name='Sales')

weekly_sales_df = (
    CDNOW_master
    .with_columns(((pl.col('PurchDay') - 1) // 7 + 1).alias('Week'))  
    .group_by('Week')
    .agg(pl.col('Quant').sum().alias('Total Sales'))
    .sort('Week')
    .filter(pl.col('Week') <= 12)
    .join(other=weeklytrierquant, on='Week', how='left')
    .with_columns((pl.col('Total Sales') - pl.col('Triers Quant')).alias('Repeat Sales'))
    .rename({'Triers Quant': 'Trial Sales'})
)

weekly_sales_df = weekly_sales_df.unpivot(index='Week', on=['Total Sales', 'Trial Sales', 'Repeat Sales'], variable_name='Actual Sales', value_name='Sales')

forecasted_chart = (
    ChartTemp(e_weekly_sales_df)
    .line_encode(y_col='Sales', y_title='Units Purchased',  color='Estimated Sales:N', x_range=12, dash=[4,4])
)

actual_chart = (
    ChartTemp(weekly_sales_df.collect())
    .line_encode(y_col='Sales', y_title='Units Purchased',  color='Actual Sales:N', x_range=12)
)

chart = forecasted_chart + actual_chart

layered_line_prop(chart=chart, title="Model Fit: Trial/Repeat Decomposition")

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [25]:
e_total_sales = e_weekly_sales
e_trial_sales = np.zeros((full_weeks.shape[0]))
e_trial_sales[:trial_weeks.shape[0]] = mean_weekly_unit_purchases.diagonal() * incr_triers
e_repeat_sales = e_total_sales - e_trial_sales

e_repeat_sales_df = pl.DataFrame({'Week': full_weeks, 
                               'Repeat Sales': e_repeat_sales})


weekly_repeat_df = (
    CDNOW_master
    .with_columns(((pl.col('PurchDay') - 1) // 7 + 1).alias('Week'))  
    .group_by('Week')
    .agg(pl.col('Quant').sum().alias('Total Sales'))
    .join(other=weeklytrierquant, on='Week', how='left')
    .fill_null(0)
    .with_columns((pl.col('Total Sales') - pl.col('Triers Quant')).alias('Repeat Sales'))
    .select('Week', 'Repeat Sales')
)

forecasted_chart = (
    ChartTemp(e_repeat_sales_df)
    .line_encode(y_col='Repeat Sales', y_title='Units Purchased', x_range=52, dash=[4,4])
)

actual_chart = (
    ChartTemp(weekly_repeat_df.filter(pl.col('Week') <= 52).collect())
    .line_encode(y_col='Repeat Sales', y_title='Units Purchased', x_range=52)
)

chart = forecasted_chart + actual_chart

layered_line_prop(chart, 'Forecast Weekly Repeat Sales')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [26]:
e_cum_repeat_sales_df = (
    e_repeat_sales_df
    .with_columns((pl.col('Repeat Sales').cum_sum()/1000).alias('Cumulative Repeat Sales'))
)

cum_weekly_repeat_df = (
    weekly_repeat_df
    .filter(pl.col('Week') <= 52)
    .sort('Week')
    .with_columns((pl.col('Repeat Sales').cum_sum()/1000).alias('Cumulative Repeat Sales'))
)

forecasted_chart = (
    ChartTemp(e_cum_repeat_sales_df)
    .line_encode(y_col='Cumulative Repeat Sales', y_title='Units Purchased (000)', x_range=52, dash=[4,4])
)

actual_chart = (
    ChartTemp(cum_weekly_repeat_df.collect())
    .line_encode(y_col='Cumulative Repeat Sales', y_title='Units Purchased (000)', x_range=52)
)

chart = forecasted_chart + actual_chart

layered_line_prop(chart, 'Cumulative Repeat Sales Forecast')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [27]:
full_weeks = np.arange(78) + 1
week_1_triers = np.zeros(78)
week_1_triers[0] = (alpha_T + beta_T - 1)/(alpha_T - 1) # expected number of units purchased in the trial week    
week_1_triers[1:] = gamma * (full_weeks[1:] - 1)**delta * beta_R / (alpha_R - 1) # expected repeat sales, number of units purchased w−i weeks after trial

rows = full_weeks.shape[0]
cols = trial_weeks.shape[0]

mean_weekly_unit_purchases = np.zeros((rows, cols))

row_indices = np.arange(rows)[:, None]
col_indices = np.arange(cols)[None, :]
lag_mask = row_indices >= col_indices

mean_weekly_unit_purchases[lag_mask] = week_1_triers[(row_indices - col_indices)[lag_mask]]
e_weekly_sales = np.sum(incr_triers * mean_weekly_unit_purchases, axis=1)

e_trial_sales = np.zeros((full_weeks.shape[0]))
e_trial_sales[:trial_weeks.shape[0]] = mean_weekly_unit_purchases.diagonal() * incr_triers
e_repeat_sales = e_weekly_sales - e_trial_sales

e_repeat_sales_df = pl.DataFrame({'Week': full_weeks,
                                  'Repeat Sales': e_repeat_sales})

forecasted_chart = (
    ChartTemp(e_repeat_sales_df)
    .line_encode(y_col='Repeat Sales', y_title='Units Purchased', x_range=78, dash=[4,4])
)

actual_chart = (
    ChartTemp(weekly_repeat_df.collect())
    .line_encode(y_col='Repeat Sales', y_title='Units Purchased', x_range=78)
)

chart = forecasted_chart + actual_chart

layered_line_prop(chart, 'Forecast Weekly Repeat Sales')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [28]:
from scipy.stats import beta

qt = np.linspace(beta.ppf(0, w1_alpha_T, w1_beta_T), beta.ppf(1, w1_alpha_T, w1_beta_T), 300)
y_w1 = beta.pdf(qt, w1_alpha_T, w1_beta_T)
y_w12 = beta.pdf(qt, allW_alpha_T, allW_beta_T)

mean_w1, var_w1 = beta.stats(w1_alpha_T, w1_beta_T, moments='mv')
mean_w1_comp = (w1_alpha_T)/(w1_alpha_T + w1_beta_T)
mean_w12, var_w12 = beta.stats(allW_alpha_T, allW_beta_T, moments='mv')
mean_w12_comp = (allW_alpha_T)/(allW_alpha_T + allW_beta_T)

display_markdown(f'''**Week 1:**
                 
$\\alpha_{{T}}$ = {w1_alpha_T:0.4f}

$\\beta_{{T}}$ = {w1_beta_T:0.4f}

Mean = {mean_w1:0.3f}, Variance = {var_w1:0.3f}

**All 12 Weeks:**

$\\alpha_{{T}}$ = {allW_alpha_T:0.4f}

$\\beta_{{T}}$ = {allW_beta_T:0.4f}

Mean = {mean_w12:0.3f}, Variance = {var_w12:0.3f}''', raw=True)

**Week 1:**
                 
$\alpha_{T}$ = 5.9117

$\beta_{T}$ = 6.2832

Mean = 0.485, Variance = 0.019

**All 12 Weeks:**

$\alpha_{T}$ = 6.9066

$\beta_{T}$ = 7.1924

Mean = 0.490, Variance = 0.017

In [ ]:
qt_dist = pl.DataFrame({'qT': qt, 'Week 1 PDF': y_w1, 'All 12 Week PDF': y_w12})
qt_dist = qt_dist.unpivot(on=['Week 1 PDF', 'All 12 Week PDF'], index='qT', variable_name='Week 1 Only vs All 12 Weeks', value_name='PDF')

(
    alt.Chart(qt_dist).mark_line()
    .encode(x=alt.X('qT', title='qₜ'), 
            y=alt.Y('PDF', title='f(qₜ)'), 
            strokeDash='Week 1 Only vs All 12 Weeks:N')
    .properties(
            width=500,
            height=350,
            title='Distribution of qₜ: Week 1 Only vs. All 12 Weeks'
        ).configure_view(stroke=None).configure_axisY(grid=False).configure_axisX(grid=False)         
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting
